# Natalia Organek - laboratorium 6

## Imports

In [1]:
import numpy as np
import pandas as pd

import os
from itertools import islice
from tqdm.notebook import tqdm
import bokeh
import bokeh.io
import bokeh.plotting
import bokeh.models
from PyAstronomy.pyasl import decimalYear


import tensorflow as tf
import tensorflow_probability as tfp

from IPython.display import display, HTML

tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels

np.random.seed(42)
tf.random.set_seed(42)

/home/oreganko/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
bokeh.plotting.output_notebook()

Loading BokehJS ...

## In situ air measurements at Mauna Loa, Observatory, Hawaii

In [3]:
manua_data = pd.read_csv('manua_loa.csv', 
    usecols=[3, 4], 
    na_values='-99.99',
    dtype=np.float64)

In [4]:
manua_data.dropna(inplace=True)

In [5]:
manua_data

,Date,CO2
2,1958.2027,315.70
3,1958.2877,317.45
4,1958.3699,317.51
6,1958.5370,315.86
7,1958.6219,314.93
...,...,...
763,2021.6219,414.34
764,2021.7068,412.90
765,2021.7890,413.55
766,2021.8740,414.82


In [6]:
fig = bokeh.plotting.figure(
    width=600, height=300, 
    x_range=(1958, 2021), y_range=(300, 420))
fig.xaxis.axis_label = 'Date'
fig.yaxis.axis_label = 'CO₂ (ppm)'
fig.add_layout(bokeh.models.Title(
    text='In situ air measurements at Mauna Loa, Observatory, Hawaii',
    text_font_style="italic"), 'above')
fig.add_layout(bokeh.models.Title(
    text='Atmospheric CO₂ concentrations', 
    text_font_size="14pt"), 'above')
fig.line(
    manua_data['Date'], manua_data['CO2'], legend_label='All data',
    line_width=2, line_color='midnightblue')
fig.legend.location = 'top_left'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)

## Train test split

In [7]:
df_observed = manua_data[manua_data['Date'] < 2012]

In [8]:
df_predict = manua_data[manua_data['Date'] >= 2012]

## Mean function

In [9]:
observations_mean = tf.constant(
    [np.mean(df_observed.CO2.values)], dtype=tf.float64)
mean_fn = lambda _: observations_mean

## Kernel function

In [10]:
constrain_positive = tfb.Shift(np.finfo(np.float64).tiny)(tfb.Exp())

# Smooth kernel hyperparameters
smooth_amplitude = tfp.util.TransformedVariable(
    initial_value=10., bijector=constrain_positive, dtype=np.float64,
    name='smooth_amplitude')
smooth_length_scale = tfp.util.TransformedVariable(
    initial_value=10., bijector=constrain_positive, dtype=np.float64,
    name='smooth_length_scale')
# Smooth kernel
smooth_kernel = tfk.ExponentiatedQuadratic(
    amplitude=smooth_amplitude, 
    length_scale=smooth_length_scale)

# Local periodic kernel hyperparameters
periodic_amplitude = tfp.util.TransformedVariable(
    initial_value=5.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_amplitude')
periodic_length_scale = tfp.util.TransformedVariable(
    initial_value=1.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_length_scale')
periodic_period = tfp.util.TransformedVariable(
    initial_value=1.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_period')
periodic_local_length_scale = tfp.util.TransformedVariable(
    initial_value=1.0, bijector=constrain_positive, dtype=np.float64,
    name='periodic_local_length_scale')
# Local periodic kernel
local_periodic_kernel = (
    tfk.ExpSinSquared(
        amplitude=periodic_amplitude, 
        length_scale=periodic_length_scale,
        period=periodic_period) * 
    tfk.ExponentiatedQuadratic(
        length_scale=periodic_local_length_scale))

# Short-medium term irregularities kernel hyperparameters
irregular_amplitude = tfp.util.TransformedVariable(
    initial_value=1., bijector=constrain_positive, dtype=np.float64,
    name='irregular_amplitude')
irregular_length_scale = tfp.util.TransformedVariable(
    initial_value=1., bijector=constrain_positive, dtype=np.float64,
    name='irregular_length_scale')
irregular_scale_mixture = tfp.util.TransformedVariable(
    initial_value=1., bijector=constrain_positive, dtype=np.float64,
    name='irregular_scale_mixture')
# Short-medium term irregularities kernel
irregular_kernel = tfk.RationalQuadratic(
    amplitude=irregular_amplitude,
    length_scale=irregular_length_scale,
    scale_mixture_rate=irregular_scale_mixture)

# Noise variance of observations
# Start out with a medium-to high noise
observation_noise_variance = tfp.util.TransformedVariable(
    initial_value=1, bijector=constrain_positive, dtype=np.float64,
    name='observation_noise_variance')

trainable_variables = [v.variables[0] for v in [
    smooth_amplitude,
    smooth_length_scale,
    periodic_amplitude,
    periodic_length_scale,
    periodic_period,
    periodic_local_length_scale,
    irregular_amplitude,
    irregular_length_scale,
    irregular_scale_mixture,
    observation_noise_variance
]]

In [11]:
kernel = (smooth_kernel + local_periodic_kernel + irregular_kernel)

## Hyperparameters tuning
### Minimizing the negative marginal log likelihood

In [12]:
batch_size = 128

batched_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (df_observed.Date.values.reshape(-1, 1), df_observed.CO2.values))
    .shuffle(buffer_size=len(df_observed))
    .repeat(count=None)
    .batch(batch_size)
)

In [13]:
@tf.function(autograph=False, experimental_compile=False)
def gp_loss_fn(index_points, observations):
    """Gaussian process negative-log-likelihood loss function."""
    gp = tfd.GaussianProcess(
        mean_fn=mean_fn,
        kernel=kernel,
        index_points=index_points,
        observation_noise_variance=observation_noise_variance
    )
    
    negative_log_likelihood = -gp.log_prob(observations)
    return negative_log_likelihood

### Adam optimizer

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Training loop
batch_nlls = []  # Batch NLL for plotting
full_ll = []  # Full data NLL for plotting
nb_iterations = 10001
for i, (index_points_batch, observations_batch) in tqdm(
        enumerate(islice(batched_dataset, nb_iterations)), total=nb_iterations):
    # Run optimization for single batch
    with tf.GradientTape() as tape:
        loss = gp_loss_fn(index_points_batch, observations_batch)
    grads = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(grads, trainable_variables))
    batch_nlls.append((i, loss.numpy()))
    # Evaluate on all observations
    if i % 100 == 0:
        # Evaluate on all observed data
        ll = gp_loss_fn(
            index_points=df_observed.Date.values.reshape(-1, 1),
            observations=df_observed.CO2.values)
        full_ll.append((i, ll.numpy()))

In [15]:
fig = bokeh.plotting.figure(
    width=600, height=350, 
    x_range=(0, nb_iterations), y_range=(50, 200))
fig.add_layout(bokeh.models.Title(
    text='Negative Log-Likelihood (NLL) during training', 
    text_font_size="14pt"), 'above')
fig.xaxis.axis_label = 'iteration'
fig.yaxis.axis_label = 'NLL batch'
# First plot
fig.line(
    *zip(*batch_nlls), legend_label='Batch data',
    line_width=2, line_color='midnightblue')
# Seoncd plot
# Setting the second y axis range name and range
fig.extra_y_ranges = {
    'fig1ax2': bokeh.models.Range1d(start=130, end=250)}
fig.line(
    *zip(*full_ll), legend_label='All observed data',
    line_width=2, line_color='red', y_range_name='fig1ax2')
# Adding the second axis to the plot.  
fig.add_layout(bokeh.models.LinearAxis(
    y_range_name='fig1ax2', axis_label='NLL all'), 'right')

fig.legend.location = 'top_right'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)

### Found hyperparameters

In [16]:
variables = [
    smooth_amplitude,
    smooth_length_scale,
    periodic_amplitude,
    periodic_length_scale,
    periodic_period,
    periodic_local_length_scale,
    irregular_amplitude,
    irregular_length_scale,
    irregular_scale_mixture,
    observation_noise_variance
]

data = list([(var.variables[0].name[:-2], var.numpy()) for var in variables])
df_variables = pd.DataFrame(
    data, columns=['Hyperparameters', 'Value'])
display(HTML(df_variables.to_html(
    index=False, float_format=lambda x: f'{x:.4f}')))
#

Hyperparameters,Value
smooth_amplitude,126.65474052393077
smooth_length_scale,99.05350796036885
periodic_amplitude,3.1113764228928824
periodic_length_scale,1.6631676762395966
periodic_period,0.999723915656506
periodic_local_length_scale,111.68380221509636
irregular_amplitude,1.0163455815432432
irregular_length_scale,1.3750762446633624
irregular_scale_mixture,0.10202147490490651
observation_noise_variance,0.05312215964969696


## Posterior predictions

In [17]:
gp_posterior_predict = tfd.GaussianProcessRegressionModel(
    mean_fn=mean_fn,
    kernel=kernel,
    index_points=df_predict.Date.values.reshape(-1, 1),
    observation_index_points=df_observed.Date.values.reshape(-1, 1),
    observations=df_observed.CO2.values,
    observation_noise_variance=observation_noise_variance)

# Posterior mean and standard deviation
posterior_mean_predict = gp_posterior_predict.mean()
posterior_std_predict = gp_posterior_predict.stddev()

In [18]:
μ = posterior_mean_predict.numpy()
σ = posterior_std_predict.numpy()

# Plot
fig = bokeh.plotting.figure(
    width=600, height=400,
    x_range=(2012, 2022), y_range=(384, 418))
fig.xaxis.axis_label = 'Date'
fig.yaxis.axis_label = 'CO₂ (ppm)'
fig.add_layout(bokeh.models.Title(
    text='Posterior predictions conditioned on observations before 2012.',
    text_font_style="italic"), 'above')
fig.add_layout(bokeh.models.Title(
    text='Atmospheric CO₂ concentrations', 
    text_font_size="14pt"), 'above')
fig.circle(
    manua_data.Date, manua_data.CO2, legend_label='True data',
    size=2, line_color='midnightblue')
fig.line(
    df_predict.Date.values, μ, legend_label='μ (predictions)',
    line_width=2, line_color='firebrick')
# Prediction interval
band_x = np.append(
    df_predict.Date.values, df_predict.Date.values[::-1])
band_y = np.append(
    (μ + 2*σ), (μ - 2*σ)[::-1])
fig.patch(
    band_x, band_y, color='firebrick', alpha=0.4, 
    line_color='firebrick', legend_label='2σ')

fig.legend.location = 'top_left'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)

### Skomentuj uzyskane wyniki. W jakim horyzoncie czasowym wyniki przewidywania koncentracji CO2 mieszczą się w przedziale niepewności ±2σ? Jaki trend można zaobserwować dla predykcji i rzeczywistych wartości od około 2016 roku?

Wartości predykcji są prawie zawsze mniejsze niż prawdziwe zmierzone wartości. Do około początku 2016 roku  (czyli po około 4 latach) prawdziwe wartości najczęściej znajdują się w obrębie niepewności predykcji, czasem są większe niż wartość predykcji wraz z niepewnością, szczególnie w pikach maksymalnej koncentracji CO2. 

Od 2016 roku te dane częściej się rozmywają, zdecydowana większość punktów pomiarowych nie mieści się w niepewności predykcji.

# Stock data

In [19]:
from datetime import datetime

def to_dec(x):
    return float(decimalYear(datetime.strptime(x, '%Y-%m-%d')))

In [20]:
def prep_stock_data(csv):
    wig_data = pd.read_csv(csv, 
        usecols=[0, 4])
    wig_data.dropna(inplace=True)
    wig_data['Data1'] = wig_data['Data'].apply(to_dec)
    return wig_data

In [21]:
def plot_all(data, name, x_range, y_range):
    fig = bokeh.plotting.figure(
        width=600, height=300, 
        x_range=x_range, y_range=y_range)
    fig.xaxis.axis_label = 'Data'
    fig.yaxis.axis_label = 'Zamkniecie'
    fig.add_layout(bokeh.models.Title(
        text=name,
        text_font_style="italic"), 'above')
    fig.add_layout(bokeh.models.Title(
        text=name, 
        text_font_size="14pt"), 'above')
    fig.line(
        data['Data1'], data['Zamkniecie'], legend_label='All data',
        line_width=2, line_color='midnightblue')
    fig.legend.location = 'top_left'
    fig.toolbar.autohide = True
    bokeh.plotting.show(fig)

In [22]:
def split_data(data, year):
    df_observed = data[data['Data1'] < year]
    df_predict = data[data['Data1'] >= year]

    return df_observed, df_predict

In [23]:
def get_mean_fn(data):
    observations_mean = tf.constant(
    [np.mean(data['Zamkniecie'].values)], dtype=tf.float64)

    return lambda _: observations_mean

In [24]:
constrain_positive = tfb.Shift(np.finfo(np.float64).tiny)(tfb.Exp())

def create_kernel():
    # Smooth kernel hyperparameters
    smooth_amplitude = tfp.util.TransformedVariable(
        initial_value=10., bijector=constrain_positive, dtype=np.float64,
        name='smooth_amplitude')
    smooth_length_scale = tfp.util.TransformedVariable(
        initial_value=10., bijector=constrain_positive, dtype=np.float64,
        name='smooth_length_scale')
    # Smooth kernel
    smooth_kernel = tfk.ExponentiatedQuadratic(
        amplitude=smooth_amplitude, 
        length_scale=smooth_length_scale)

    # Local periodic kernel hyperparameters
    periodic_amplitude = tfp.util.TransformedVariable(
        initial_value=5.0, bijector=constrain_positive, dtype=np.float64,
        name='periodic_amplitude')
    periodic_length_scale = tfp.util.TransformedVariable(
        initial_value=1.0, bijector=constrain_positive, dtype=np.float64,
        name='periodic_length_scale')
    periodic_period = tfp.util.TransformedVariable(
        initial_value=1.0, bijector=constrain_positive, dtype=np.float64,
        name='periodic_period')
    periodic_local_length_scale = tfp.util.TransformedVariable(
        initial_value=1.0, bijector=constrain_positive, dtype=np.float64,
        name='periodic_local_length_scale')
    # Local periodic kernel
    local_periodic_kernel = (
        tfk.ExpSinSquared(
            amplitude=periodic_amplitude, 
            length_scale=periodic_length_scale,
            period=periodic_period) * 
        tfk.ExponentiatedQuadratic(
            length_scale=periodic_local_length_scale))

    # Short-medium term irregularities kernel hyperparameters
    irregular_amplitude = tfp.util.TransformedVariable(
        initial_value=1., bijector=constrain_positive, dtype=np.float64,
        name='irregular_amplitude')
    irregular_length_scale = tfp.util.TransformedVariable(
        initial_value=1., bijector=constrain_positive, dtype=np.float64,
        name='irregular_length_scale')
    irregular_scale_mixture = tfp.util.TransformedVariable(
        initial_value=1., bijector=constrain_positive, dtype=np.float64,
        name='irregular_scale_mixture')
    # Short-medium term irregularities kernel
    irregular_kernel = tfk.RationalQuadratic(
        amplitude=irregular_amplitude,
        length_scale=irregular_length_scale,
        scale_mixture_rate=irregular_scale_mixture)
    
    
    trainable_variables = [v.variables[0] for v in [
        smooth_amplitude,
        smooth_length_scale,
        periodic_amplitude,
        periodic_length_scale,
        periodic_period,
        periodic_local_length_scale,
        irregular_amplitude,
        irregular_length_scale,
        irregular_scale_mixture,
#         observation_noise_variance
    ]]

    return smooth_kernel + local_periodic_kernel + irregular_kernel, trainable_variables

In [25]:
def batch_dataset(data, batch_size = 128):
    return (tf.data.Dataset.from_tensor_slices(
        (data.Data1.values.reshape(-1, 1), data.Zamkniecie.values))
        .shuffle(buffer_size=len(data))
        .repeat(count=None)
        .batch(batch_size))

def get_observation_noise_variance(trainable_variables):
    onv =  tfp.util.TransformedVariable(
        initial_value=1, bijector=constrain_positive, dtype=np.float64,
        name='observation_noise_variance')
    trainable_variables.append(onv.variables[0])
    return onv

In [26]:
@tf.function(autograph=False, experimental_compile=False)
def gp_loss_fn(index_points, observations, mean_fn, kernel, observation_noise_variance):
    """Gaussian process negative-log-likelihood loss function."""
    gp = tfd.GaussianProcess(
        mean_fn=mean_fn,
        kernel=kernel,
        index_points=index_points,
        observation_noise_variance=observation_noise_variance
    )
    
    negative_log_likelihood = -gp.log_prob(observations)
    return negative_log_likelihood


def train(batched_dataset, data, mean_fn, kernel, observation_noise_variance, trainable_variables, nb_iterations = 10001):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Training loop
    batch_nlls = []  # Batch NLL for plotting
    full_ll = []  # Full data NLL for plotting
    for i, (index_points_batch, observations_batch) in tqdm(
            enumerate(islice(batched_dataset, nb_iterations)), total=nb_iterations):
        # Run optimization for single batch
        with tf.GradientTape() as tape:
            loss = gp_loss_fn(index_points_batch, observations_batch, mean_fn, kernel, observation_noise_variance)
        grads = tape.gradient(loss, trainable_variables)
        optimizer.apply_gradients(zip(grads, trainable_variables))
        batch_nlls.append((i, loss.numpy()))
        # Evaluate on all observations
        if i % 100 == 0:
            # Evaluate on all observed data
            ll = gp_loss_fn(
                index_points=data.Data1.values.reshape(-1, 1),
                observations=data.Zamkniecie.values, 
                mean_fn=mean_fn, kernel=kernel, observation_noise_variance=observation_noise_variance)
            full_ll.append((i, ll.numpy()))
    return batch_nlls, full_ll

In [27]:
def print_learning(batch_nlls, full_ll, nb_iterations = 10001):
    fig = bokeh.plotting.figure(
    width=600, height=350, 
    x_range=(0, nb_iterations), y_range=(0, 100000))
    fig.add_layout(bokeh.models.Title(
        text='Negative Log-Likelihood (NLL) during training', 
        text_font_size="14pt"), 'above')
    fig.xaxis.axis_label = 'iteration'
    fig.yaxis.axis_label = 'NLL batch'
    # First plot
    fig.line(
        *zip(*batch_nlls), legend_label='Batch data',
        line_width=2, line_color='midnightblue')
    # Seoncd plot
    # Setting the second y axis range name and range
    fig.extra_y_ranges = {
        'fig1ax2': bokeh.models.Range1d(start=100000, end=5000000)}
    fig.line(
        *zip(*full_ll), legend_label='All observed data',
        line_width=2, line_color='red', y_range_name='fig1ax2')
    # Adding the second axis to the plot.  
    fig.add_layout(bokeh.models.LinearAxis(
        y_range_name='fig1ax2', axis_label='NLL all'), 'right')

    fig.legend.location = 'top_right'
    fig.toolbar.autohide = True
    bokeh.plotting.show(fig)

In [28]:
def plot_predictions(data, mean_fn, kernel, df_predict, df_observed, observation_noise_variance,
                    x_range, y_range):
    gp_posterior_predict = tfd.GaussianProcessRegressionModel(
    mean_fn=mean_fn,
    kernel=kernel,
    index_points=df_predict.Data1.values.reshape(-1, 1),
    observation_index_points=df_observed.Data1.values.reshape(-1, 1),
    observations=df_observed.Zamkniecie.values,
    observation_noise_variance=observation_noise_variance)

    # Posterior mean and standard deviation
    posterior_mean_predict = gp_posterior_predict.mean()
    posterior_std_predict = gp_posterior_predict.stddev()
    
    μ = posterior_mean_predict.numpy()
    σ = posterior_std_predict.numpy()

    # Plot
    fig = bokeh.plotting.figure(
        width=600, height=400,
        x_range=x_range, y_range=y_range)
    fig.xaxis.axis_label = 'Data'
    fig.yaxis.axis_label = 'Zamknięcie'
    fig.add_layout(bokeh.models.Title(
        text='Posterior predictions conditioned on observations.',
        text_font_style="italic"), 'above')
    fig.add_layout(bokeh.models.Title(
        text='Zamknięcie', 
        text_font_size="14pt"), 'above')
    fig.circle(
        data.Data1, data.Zamkniecie, legend_label='True data',
        size=2, line_color='midnightblue')
    fig.line(
        df_predict.Data1.values, μ, legend_label='μ (predictions)',
        line_width=2, line_color='firebrick')
    # Prediction interval
    band_x = np.append(
        df_predict.Data1.values, df_predict.Data1.values[::-1])
    band_y = np.append(
        (μ + 2*σ), (μ - 2*σ)[::-1])
    fig.patch(
        band_x, band_y, color='firebrick', alpha=0.4, 
        line_color='firebrick', legend_label='2σ')

    fig.legend.location = 'top_left'
    fig.toolbar.autohide = True
    bokeh.plotting.show(fig)

In [29]:
days_7 = .0192
days_14 = .0383

In [30]:
wig_data = prep_stock_data('wig20_d.csv')
plot_all(wig_data, 'Wig 20', (1991, 2021), (0, 4000))

In [31]:
df_observed, df_predict = split_data(wig_data, 2019)

In [32]:
constrain_positive = tfb.Shift(np.finfo(np.float64).tiny)(tfb.Exp())
mean_fn = get_mean_fn(df_observed)
kernel, trainable_variables = create_kernel()
observation_noise_variance = get_observation_noise_variance(trainable_variables)

batched_dataset = batch_dataset(df_observed)
batch_nlls, full_ll = train(batched_dataset, df_observed, mean_fn, kernel, observation_noise_variance, trainable_variables)


In [33]:
print_learning(batch_nlls, full_ll)

### Prediction for 7 days

In [34]:
plot_predictions(wig_data, mean_fn, kernel, df_predict, df_observed, observation_noise_variance,
                 (2019, 2019 + days_7), (2000, 2500))

### Prediction for 14 days

In [35]:
plot_predictions(wig_data, mean_fn, kernel, df_predict, df_observed, observation_noise_variance,
                 (2019, 2019 + days_14), (1800, 3000))

## Split in 2020

In [36]:
split_point = 2020.18

In [37]:
df_observed, df_predict = split_data(wig_data, split_point)

In [38]:
constrain_positive = tfb.Shift(np.finfo(np.float64).tiny)(tfb.Exp())
mean_fn = get_mean_fn(df_observed)
kernel, trainable_variables = create_kernel()
observation_noise_variance = get_observation_noise_variance(trainable_variables)

batched_dataset = batch_dataset(df_observed)
batch_nlls, full_ll = train(batched_dataset, df_observed, mean_fn, kernel, observation_noise_variance, trainable_variables)


In [39]:
print_learning(batch_nlls, full_ll)

### 7 days prediction

In [40]:
plot_predictions(wig_data, mean_fn, kernel, df_predict, df_observed, observation_noise_variance,
                 (split_point, split_point + days_7), (1000, 2000))

### 14 days prediction

In [41]:
plot_predictions(wig_data, mean_fn, kernel, df_predict, df_observed, observation_noise_variance,
                 (split_point, split_point + days_14), (1000, 3000))

## CDP Red

In [42]:
cdp_data = prep_stock_data('cdr_d.csv')

In [43]:
plot_all(cdp_data, 'CD Project Red', (1994, 2022), (0, 500))

### Split in 2019

In [44]:
split_point = 2019

In [45]:
df_observed, df_predict = split_data(cdp_data, split_point)

In [46]:
mean_fn = get_mean_fn(df_observed)
kernel, trainable_variables = create_kernel()
observation_noise_variance = get_observation_noise_variance(trainable_variables)

batched_dataset = batch_dataset(df_observed)
batch_nlls, full_ll = train(batched_dataset, df_observed, mean_fn, kernel, observation_noise_variance, trainable_variables)


In [47]:
def print_learning(batch_nlls, full_ll, nb_iterations = 10001):
    fig = bokeh.plotting.figure(
    width=600, height=350, 
    x_range=(0, nb_iterations), y_range=(300, 1500))
    fig.add_layout(bokeh.models.Title(
        text='Negative Log-Likelihood (NLL) during training', 
        text_font_size="14pt"), 'above')
    fig.xaxis.axis_label = 'iteration'
    fig.yaxis.axis_label = 'NLL batch'
    # First plot
    fig.line(
        *zip(*batch_nlls), legend_label='Batch data',
        line_width=2, line_color='midnightblue')
    # Seoncd plot
    # Setting the second y axis range name and range
    fig.extra_y_ranges = {
        'fig1ax2': bokeh.models.Range1d(start=10000, end=20000)}
    fig.line(
        *zip(*full_ll), legend_label='All observed data',
        line_width=2, line_color='red', y_range_name='fig1ax2')
    # Adding the second axis to the plot.  
    fig.add_layout(bokeh.models.LinearAxis(
        y_range_name='fig1ax2', axis_label='NLL all'), 'right')

    fig.legend.location = 'top_right'
    fig.toolbar.autohide = True
    bokeh.plotting.show(fig)

In [48]:
print_learning(batch_nlls, full_ll)

### 7 days prediction

In [54]:
plot_predictions(cdp_data, mean_fn, kernel, df_predict, df_observed, observation_noise_variance,
                 (split_point, split_point + days_7), (140, 180))

### 14 days prediction

In [55]:
plot_predictions(cdp_data, mean_fn, kernel, df_predict, df_observed, observation_noise_variance,
                 (split_point, split_point + days_14), (140, 180))

## Split in 2020

In [56]:
split_point = 2020.15

In [57]:
df_observed, df_predict = split_data(cdp_data, split_point)

In [58]:
mean_fn = get_mean_fn(df_observed)
kernel, trainable_variables = create_kernel()
observation_noise_variance = get_observation_noise_variance(trainable_variables)

batched_dataset = batch_dataset(df_observed)
batch_nlls, full_ll = train(batched_dataset, df_observed, mean_fn, kernel, observation_noise_variance, trainable_variables)


In [59]:
def print_learning(batch_nlls, full_ll, nb_iterations = 10001):
    fig = bokeh.plotting.figure(
    width=600, height=350, 
    x_range=(0, nb_iterations), y_range=(300, 1500))
    fig.add_layout(bokeh.models.Title(
        text='Negative Log-Likelihood (NLL) during training', 
        text_font_size="14pt"), 'above')
    fig.xaxis.axis_label = 'iteration'
    fig.yaxis.axis_label = 'NLL batch'
    # First plot
    fig.line(
        *zip(*batch_nlls), legend_label='Batch data',
        line_width=2, line_color='midnightblue')
    # Seoncd plot
    # Setting the second y axis range name and range
    fig.extra_y_ranges = {
        'fig1ax2': bokeh.models.Range1d(start=10000, end=20000)}
    fig.line(
        *zip(*full_ll), legend_label='All observed data',
        line_width=2, line_color='red', y_range_name='fig1ax2')
    # Adding the second axis to the plot.  
    fig.add_layout(bokeh.models.LinearAxis(
        y_range_name='fig1ax2', axis_label='NLL all'), 'right')

    fig.legend.location = 'top_right'
    fig.toolbar.autohide = True
    bokeh.plotting.show(fig)

In [60]:
print_learning(batch_nlls, full_ll)

### 7 days prediction

In [64]:
plot_predictions(cdp_data, mean_fn, kernel, df_predict, df_observed, observation_noise_variance,
                 (split_point, split_point + days_7), (250, 450))

### 14 days prediction

In [66]:
plot_predictions(cdp_data, mean_fn, kernel, df_predict, df_observed, observation_noise_variance,
                 (split_point, split_point + days_14), (250, 450))

Przewidywanie giełdy daje znacznie gorsze wyniki. W zależności od podziału predykcja jest mniej lub bardziej dokładna, jednak notowania giełdowe nie mają regularnych wartości (jak w przypadku danych z Manua Loa, gdzie tendencja była wzrostowa + cyklicznie rosnąca i malejąca).  


Dane wig20 na przestrzeni lat mają bardzo duże wahania, w porównaniu z nimi dane z 2019 roku mają bardzo małą amplitudę. Wahania z okresu 7/14 dni nie mieszczą się w niepewności (kilka pierwszych punktów mieści się, ale już pik w 2-gim tygodniu nie). 

W 2020 można zaobserwować gwałtowny spadek na giełdzie. Model co prawda uchwycił ten spadek, tendencja jest malejąca, ale prawdziwe dane oscylują wokół predykcji (przez ok. 2 tygodnie), w większości jednak nie zawierają się w niepewności modelu.


Dane CDR różnią się od wig20 - od ok. 2015 roku mają stałą tendencję wzrostową z nieregularnymi wahaniami o zwykle niewielkiej amplitudzie i kilkoma wyraźnymi spadkami (np 2020 i 2021 rok).

Podział na początku 2019 roku skutkuje całkiem dobrą predykcją w okresie 7 dni i trochę gorszą w 2-gim tygodniu (prawdziwe dane zaczęły szybciej rosnąć).

Podział w 2020 roku (starałam się uchwycić moment spadku) daje gorsze predykcje - model dalej pokazuje wzrost, podczas gdy prawdziwe dane są daleko poniżej niepewności.

Wygląda to tak, jakby model prederował informacje o globalnym, długoterminowym wzroście/spadku niż krótkotrwałe wahania (które nie są regularne, więc ciężko jest je przewidywać).

